**Ini catatan untuk belajar forecasting di SQLServer**

buat database jika tidak ada

In [20]:

USE master
GO
-- Create the new database if it does not exist already
IF NOT EXISTS (
    SELECT [name]
        FROM sys.databases
        WHERE [name] = N'belajar_statistik'
)
CREATE DATABASE belajar_statistik
GO

Use belajar_statistik
go

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.163

buat table data dan isinya

In [21]:
IF OBJECT_ID('[dbo].[data_timeseries]', 'U') IS NOT NULL
DROP TABLE [dbo].[data_timeseries]
GO
-- Create the table in the specified schema
CREATE TABLE [dbo].[data_timeseries]
(
    [Id] INT NOT NULL PRIMARY KEY, -- Primary Key column
    [data] INT NOT NULL
);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.026

In [22]:
INSERT INTO [dbo].[data_timeseries](ID,data) values (1,3);
INSERT INTO [dbo].[data_timeseries](ID,data) values (2,10);
INSERT INTO [dbo].[data_timeseries](ID,data) values (3,15);
INSERT INTO [dbo].[data_timeseries](ID,data) values (4,20);
INSERT INTO [dbo].[data_timeseries](ID,data) values (5,3);
INSERT INTO [dbo].[data_timeseries](ID,data) values (6,5);
INSERT INTO [dbo].[data_timeseries](ID,data) values (7,10);

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.056

In [23]:
SELECT *
FROM data_timeseries;

(7 rows affected)

Total execution time: 00:00:00.023

Id,data
1,3
2,10
3,15
4,20
5,3
6,5
7,10


**Naive method**,  
jika misalnya kita membuat model   
data peramalan kita adalah data observasi terakir + 1

In [24]:
-- dapatkan data terakir 

SELECT top 1 *
FROM data_timeseries 
ORDER BY id DESC;

(1 row affected)

Total execution time: 00:00:00.020

Id,data
7,10


In [25]:
-- tambahkan data terakhir  + 1
SELECT data + 1 as hasil_forecast
FROM (
    SELECT top 1 *
    FROM data_timeseries 
    ORDER BY id DESC
) a;

(1 row affected)

Total execution time: 00:00:00.008

hasil_forecast
11


**Metode Rata2**

  

nilai forecast bisa juga didapatkan dari melakukan rata2 dari data

In [26]:
SELECT avg(data) as hasil_forecast
from data_timeseries;

(1 row affected)

Total execution time: 00:00:00.011

hasil_forecast
9


**Metode moving average** 

  

moving average seperti merata2 data namun hanya sebagian data, bisanya beberapa data terakhir

In [27]:
-- misalnya dengan timewindow 5, maka kita akan merata2 data 5 terakhir 

-- dapatkan 5 data terakhir 

SELECT top 5 *
FROM data_timeseries 
ORDER BY ID DESC ;

(5 rows affected)

Total execution time: 00:00:00.009

Id,data
7,10
6,5
5,3
4,20
3,15


In [28]:
-- hitunh rata2 5 data terakhir 
SELECT avg(data) as hasil_forecast
FROM (
    SELECT top 5 *
    FROM data_timeseries 
    ORDER BY ID DESC 
) a;

(1 row affected)

Total execution time: 00:00:00.013

hasil_forecast
10


**Metode Weighted Moving Average**

seperti moving average, namun ada pembobotan di tiap point data , total bobot haruslah 1

In [29]:
-- buat data bobot 
;WITH bobot as (
    SELECT 1 as point_ke, 0.25 as bobot 
    UNION ALL 
    SELECT 2 as point_ke, 0.25 as bobot
    UNION ALL
    SELECT 3 as point_ke, 0.25 as bobot
    UNION ALL 
    SELECT 4 as point_ke, 0.20 as bobot
    UNION ALL 
    SELECT 5 as point_ke, 0.05 as bobot
) 
SELECT * FROM bobot;

(5 rows affected)

Total execution time: 00:00:00.010

point_ke,bobot
1,0.25
2,0.25
3,0.25
4,0.20
5,0.05


In [30]:
-- memastikan bobotnya bertotal 1 

;WITH bobot as (
    SELECT 1 as point_ke, 0.25 as bobot 
    UNION ALL 
    SELECT 2 as point_ke, 0.25 as bobot
    UNION ALL
    SELECT 3 as point_ke, 0.25 as bobot
    UNION ALL 
    SELECT 4 as point_ke, 0.20 as bobot
    UNION ALL 
    SELECT 5 as point_ke, 0.05 as bobot
) 
SELECT sum(bobot) as total_bobot FROM bobot;

(1 row affected)

Total execution time: 00:00:00.010

total_bobot
1.00


In [31]:
-- ambil 5 data terakhir dan sematkan id dengan urutan yang terbaru duluan 
SELECT *
FROM (
    SELECT ROW_NUMBER() over(order by ID desc) as rn,data
    FROM data_timeseries
) a
WHERE rn <= 5;

(5 rows affected)

Total execution time: 00:00:00.009

rn,data
1,10
2,5
3,3
4,20
5,15


In [32]:
-- gabungkan bobot dan data  serta diproses 
;WITH bobot as (
    SELECT 1 as point_ke, 0.25 as bobot 
    UNION ALL 
    SELECT 2 as point_ke, 0.25 as bobot
    UNION ALL
    SELECT 3 as point_ke, 0.25 as bobot
    UNION ALL 
    SELECT 4 as point_ke, 0.20 as bobot
    UNION ALL 
    SELECT 5 as point_ke, 0.05 as bobot
), data_final as (

    SELECT *
    FROM (
        SELECT ROW_NUMBER() over(order by ID desc) as rn,data
        FROM data_timeseries
    ) a
    WHERE rn <= 5
)
SELECT a.rn,a.data,b.bobot,a.data * b.bobot as nilai_terbobot
FROM data_final a 
JOIN bobot b ON a.rn = b.point_ke

(5 rows affected)

Total execution time: 00:00:00.011

rn,data,bobot,nilai_terbobot
1,10,0.25,2.50
2,5,0.25,1.25
3,3,0.25,0.75
4,20,0.20,4.00
5,15,0.05,0.75


In [33]:
-- lakukan perhitungan forecast 
;WITH bobot as (
    SELECT 1 as point_ke, 0.25 as bobot 
    UNION ALL 
    SELECT 2 as point_ke, 0.25 as bobot
    UNION ALL
    SELECT 3 as point_ke, 0.25 as bobot
    UNION ALL 
    SELECT 4 as point_ke, 0.20 as bobot
    UNION ALL 
    SELECT 5 as point_ke, 0.05 as bobot
), data_final as (

    SELECT *
    FROM (
        SELECT ROW_NUMBER() over(order by ID desc) as rn,data
        FROM data_timeseries
    ) a
    WHERE rn <= 5
), nilai_terbobot  as(
    SELECT a.rn,a.data,b.bobot,a.data * b.bobot as nilai_terbobot
    FROM data_final a 
    JOIN bobot b ON a.rn = b.point_ke
)
SELECT sum(nilai_terbobot) as nilai_forecast
FROM nilai_terbobot

(1 row affected)

Total execution time: 00:00:00.014

nilai_forecast
9.25
